## 2. Imputing, feature selection and feature generation - target
### 2.1 Imputing target using KNN


In [118]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [119]:
data_raw = pd.read_csv('perenials_dataset1.csv', encoding='utf8', header=0, delimiter=';')
data_raw.head()

,ID,IDstudy,plotID,country,region,temperature_Celsiul,precipitation_mm,climate,Latitud,Longitud,bedrock,soil_type,USDA,TYPE_CHANGE,CROP_current,CROP_type,cropAGE,current_land_use,LUC_this_study,previous_land_use,years_since_luc,TEMPORAL_only_this study,timeTEMPORAL,CHANGE_management,year_since_MANAGEMENT_CHANGE,soil_treatments,tillageYESNO,tillage_type,fertilizerYESNO,fet_type,rate_fertilizer_kh_ha,pesticides,org_amendYESNO,org_amend_type,prev_man,SINGLE_DATA-MULTIPLE_SINGLEAMPLING,DIF_DEPTH_current_point,CONTI_DEPTH,NUM_DEPTH,year_measure,N_plots_current,N_sample_per_plot_current,N_measured_current,soil_from_cm_current,soil_to_cm_current,depth_midpoint_current,bulk_density_Mg_m3_current,%clay_current,%silt_current,%sand_current,ph_current,SOC_g_kg_current,sd_SOC_gkg_current,SOC_Mg_ha_current,sd_SOC_Mgha_current,N_plots_previous,N_sample_per_plot_previous,N_measured_current.1,soil_from_cm_previous,soil_to_cm_previous,depth_midpoint_previous,Bulkdensity_previous,%clay_previous,%silt_previous,%sand_previous,ph_previous,SOC_g_kg_previous,sd_SOC_gkg_previous,SOC_Mg_ha_previous,sd_SOC_Mgha_previous,yearPpub
0,1,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,0.0,10.0,5.0,1.30,13.7,5.1,81.2,NaN,NaN,NaN,20.70,NaN,20.0,1.0,20,0.0,10.0,5.0,1.36,19.2,10.4,70.5,NaN,NaN,NaN,17.39,NaN,2017.0
1,2,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,10.0,20.0,15.0,1.34,14.8,1.1,84.2,NaN,NaN,NaN,12.54,NaN,20.0,1.0,20,10.0,20.0,15.0,1.58,21.7,8.9,69.3,NaN,NaN,NaN,15.57,NaN,2017.0
2,3,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,20.0,60.0,40.0,1.55,18.0,2.0,80.0,NaN,NaN,NaN,30.92,NaN,20.0,1.0,20,20.0,60.0,40.0,1.41,24.8,8.7,66.5,NaN,NaN,NaN,55.38,NaN,2017.0
3,4,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,60.0,100.0,80.0,1.46,18.8,3.5,77.7,NaN,NaN,NaN,36.23,NaN,20.0,1.0,20,60.0,100.0,80.0,1.31,24.8,9.2,66.0,NaN,NaN,NaN,36.10,NaN,2017.0
4,5,1,2,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Luvisol,Ultisol,LUC,Sugarcane,grass,4.0,bioenergy,YES,perennial_food,4.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,0.0,10.0,5.0,1.20,38.1,10.9,51.0,NaN,NaN,NaN,16.43,NaN,20.0,1.0,20,0.0,10.0,5.0,1.27,42.9,11.4,45.7,NaN,NaN,NaN,16.28,NaN,2017.0


### 2.1.1 Change the column names

In [120]:
new_cols = data_raw.columns
new_cols = [col.lower().replace (" ", "_") for col in cols]
new_cols = [col.lower().replace ("temperature_celsiul", "temp_c") for col in new_cols]
new_cos = [col.lower().replace ("precipitation_mm", "precip_c") for col in new_cols]
new_cols = [col.lower().replace ("temporal_only_this_study", "temporal") for col in new_cols]
new_cols = [col.lower().replace ("single_data-multiple_singleampling", "multiple_sampl") for col in new_cols]
new_cols = [col.lower().replace ('bulk_density_mg_m3', "bulk_density") for col in new_cols]
new_cols = [col.lower().replace ('rate_fertilizer_kh_ha', "rate_fert") for col in new_cols]

data_raw.columns = new_cols
data_raw.head()

,id,idstudy,plotid,country,region,temp_c,precipitation_mm,climate,latitud,longitud,bedrock,soil_type,usda,type_change,crop_current,crop_type,cropage,current_land_use,luc_this_study,previous_land_use,years_since_luc,temporal,timetemporal,change_management,year_since_management_change,soil_treatments,tillageyesno,tillage_type,fertilizeryesno,fet_type,rate_fert,pesticides,org_amendyesno,org_amend_type,prev_man,multiple_sampl,dif_depth_current_point,conti_depth,num_depth,year_measure,n_plots_current,n_sample_per_plot_current,n_measured_current,soil_from_cm_current,soil_to_cm_current,depth_midpoint_current,bulk_density_current,%clay_current,%silt_current,%sand_current,ph_current,soc_g_kg_current,sd_soc_gkg_current,soc_mg_ha_current,sd_soc_mgha_current,n_plots_previous,n_sample_per_plot_previous,n_measured_current.1,soil_from_cm_previous,soil_to_cm_previous,depth_midpoint_previous,bulkdensity_previous,%clay_previous,%silt_previous,%sand_previous,ph_previous,soc_g_kg_previous,sd_soc_gkg_previous,soc_mg_ha_previous,sd_soc_mgha_previous,yearppub
0,1,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,0.0,10.0,5.0,1.30,13.7,5.1,81.2,NaN,NaN,NaN,20.70,NaN,20.0,1.0,20,0.0,10.0,5.0,1.36,19.2,10.4,70.5,NaN,NaN,NaN,17.39,NaN,2017.0
1,2,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,10.0,20.0,15.0,1.34,14.8,1.1,84.2,NaN,NaN,NaN,12.54,NaN,20.0,1.0,20,10.0,20.0,15.0,1.58,21.7,8.9,69.3,NaN,NaN,NaN,15.57,NaN,2017.0
2,3,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,20.0,60.0,40.0,1.55,18.0,2.0,80.0,NaN,NaN,NaN,30.92,NaN,20.0,1.0,20,20.0,60.0,40.0,1.41,24.8,8.7,66.5,NaN,NaN,NaN,55.38,NaN,2017.0
3,4,1,1,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Ferralsols,Oxisols,LUC,Sugarcane,grass,3.0,bioenergy,YES,perennial_food,3.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,60.0,100.0,80.0,1.46,18.8,3.5,77.7,NaN,NaN,NaN,36.23,NaN,20.0,1.0,20,60.0,100.0,80.0,1.31,24.8,9.2,66.0,NaN,NaN,NaN,36.10,NaN,2017.0
4,5,1,2,Brazil,SaoPaulo,21.0,1500.0,Tropical,-21.35,-47.066,Sandstone,Luvisol,Ultisol,LUC,Sugarcane,grass,4.0,bioenergy,YES,perennial_food,4.0,NO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MULTI,YES,YES,4,2013.0,20.0,1.0,20,0.0,10.0,5.0,1.20,38.1,10.9,51.0,NaN,NaN,NaN,16.43,NaN,20.0,1.0,20,0.0,10.0,5.0,1.27,42.9,11.4,45.7,NaN,NaN,NaN,16.28,NaN,2017.0


### 2.1.2 Generating small dataframe for KNN Imputing

we onbly use some of the variable sfor KNN so, that we limit the bias. Most improtant we use the SOC g/kg together with soil type

In [121]:
data_imputer = data_raw[['id','temp_c', 'precipitation_mm','depth_midpoint_previous','depth_midpoint_current', 'soc_g_kg_previous','soc_g_kg_current', 'soc_mg_ha_previous', 'soc_mg_ha_current']].copy()

In [122]:
def display_missing(df):
    """shows the number of missing values of each column for a date_frame, if there is one"""
    for col in df.columns.tolist():
        if df[col].isnull().sum():
            print('{} column missing values: {}/{}'.format(col, df[col].isnull().sum(), len(df)))
    print ('Done checking for missing values')
display_missing(data_imputer)

temp_c column missing values: 13/1605
precipitation_mm column missing values: 9/1605
soc_g_kg_previous column missing values: 1047/1605
soc_g_kg_current column missing values: 1056/1605
soc_mg_ha_previous column missing values: 467/1605
soc_mg_ha_current column missing values: 467/1605
Done checking for missing values


### 2.1.3 Importing and proceeding climate and soil data
We calculated in xls already the averages and droped all columns with data for depth below 30 cm as we only want to predict SOC change in hte upper layer. 

In [123]:
data_clso_raw = pd.read_csv('perenials_soil_climate.csv', encoding='utf8', header=0, delimiter=';')

In [124]:
data_clso = data_clso_raw.iloc[:,:20]
data_clso.drop(['Prec_Jan'], axis=1, inplace=True)
data_clso.tail()  

,ID,Latitud,Longitud,Elevation,Slope,SLOPECLASS,Aspect,Hillshade,Rougness,CWD,PT10,avg percipitation,bd_avg,ph_avg,cat_avg,clay_avg,slit_avg,sand_avg,gridsoc_g_kg_avg
1600,1601.0,21.54,101.16,587.0,90.0,6.0,258.93,42.42,155.94,-316.58,0.01,11.00,1299.67,53.50,18.00,31.50,30.75,37.75,20.75
1601,1602.0,21.56,101.14,795.0,90.0,6.0,66.63,0.00,292.55,-316.58,0.01,12.00,1250.00,53.50,20.75,32.75,29.75,37.50,21.75
1602,1603.0,21.56,101.14,795.0,90.0,6.0,66.63,0.00,292.55,-316.58,0.01,12.00,1250.00,53.50,20.75,32.75,29.75,37.50,21.75
1603,1604.0,21.56,101.14,795.0,90.0,6.0,66.63,0.00,292.55,-316.58,0.01,12.00,1250.00,53.50,20.75,32.75,29.75,37.50,21.75
1604,1605.0,21.56,101.14,795.0,90.0,6.0,66.63,0.00,292.55,-316.58,0.01,12.00,1250.00,53.50,20.75,32.75,29.75,37.50,21.75


In [125]:
data_clso.shape

(1605, 19)

In [126]:
data_clso.dtypes
to_numeric_list = ['avg percipitation',
'bd_avg',
'ph_avg',
'cat_avg',
'clay_avg',
'slit_avg',
'sand_avg',
'gridsoc_g_kg_avg'] 
for col in to_numeric_list:
  data_clso[col] = pd.to_numeric(data_clso[col],errors='coerce')
data_clso.dtypes


ID                   float64
Latitud              float64
Longitud             float64
Elevation            float64
Slope                float64
SLOPECLASS           float64
Aspect               float64
Hillshade            float64
Rougness             float64
CWD                  float64
PT10                 float64
avg percipitation    float64
bd_avg               float64
ph_avg               float64
cat_avg              float64
clay_avg             float64
slit_avg             float64
sand_avg             float64
gridsoc_g_kg_avg     float64
dtype: object

In [127]:
data_imputer.shape

(1605, 9)

In [128]:
data_Knn = pd.concat([data_imputer, data_clso], axis = 1)

In [129]:
data_Knn.shape

(1605, 28)

### 2.1.3 Imputing with KNN 
we onluy use climate data, soil data, depth and the respective (other) SOC to run the KNN. We hereby limit the bias. 
Observations (rows) below 30 cm will be droped later on in order to make sure to generate a df with all observations and all features, merging with the original after dropping observation may lead to errors...


In [130]:
data_Knn_columns = data_Knn.columns
data_Knn_array = data_Knn.to_numpy()

#n = 3 from precious observations
Knn = KNNImputer(n_neighbors=3)

data_Knn_fill_array = Knn.fit_transform(data_Knn_array)

data_imputed = pd.DataFrame(data=data_Knn_fill_array, columns=data_Knn_columns)
data_imputed.head()

,id,temp_c,precipitation_mm,depth_midpoint_previous,depth_midpoint_current,soc_g_kg_previous,soc_g_kg_current,soc_mg_ha_previous,soc_mg_ha_current,ID,Latitud,Longitud,Elevation,Slope,SLOPECLASS,Aspect,Hillshade,Rougness,CWD,PT10,avg percipitation,bd_avg,ph_avg,cat_avg,clay_avg,slit_avg,sand_avg,gridsoc_g_kg_avg
0,1.0,21.0,1500.0,5.0,5.0,48.393333,48.840000,17.39,20.70,1.0,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5
1,2.0,21.0,1500.0,15.0,15.0,43.580000,43.373333,15.57,12.54,2.0,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5
2,3.0,21.0,1500.0,40.0,40.0,43.580000,43.373333,55.38,30.92,3.0,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5
3,4.0,21.0,1500.0,80.0,80.0,36.720000,35.653333,36.10,36.23,4.0,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5
4,5.0,21.0,1500.0,5.0,5.0,48.393333,48.840000,16.28,16.43,5.0,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5


### 

### 2.1.4 Feature engineering target

soc_mg_ha had less missing vlaues, so we rather use this. However, depending on the predictions we want to do we might want to have both measurements.



In [131]:
data_imputed['soc_change_mg'] = data_imputed['soc_mg_ha_current'] - data_imputed['soc_mg_ha_previous']
data_imputed['soc_change_gkg'] = data_imputed['soc_g_kg_current'] - data_imputed['soc_g_kg_previous']
data_imputed.shape

(1605, 30)

### 2.1.5 Cleaning the DataFrame from unecessary features and generating df to export

In [132]:
imp_cols = data_imputed.columns

In [133]:

imp_cols = [col.lower().replace (" ", "_") for col in imp_cols]
data_imputed.columns = imp_cols

In [134]:
data_soc_mg = data_imputed.copy()
data_soc_mg.drop(['soc_mg_ha_current', 'soc_g_kg_current', "soc_mg_ha_current", 'id', "soc_change_gkg"], axis=1, inplace =True)


In [135]:
data_soc_mg.head()

,temp_c,precipitation_mm,depth_midpoint_previous,depth_midpoint_current,soc_g_kg_previous,soc_mg_ha_previous,latitud,longitud,elevation,slope,slopeclass,aspect,hillshade,rougness,cwd,pt10,avg_percipitation,bd_avg,ph_avg,cat_avg,clay_avg,slit_avg,sand_avg,gridsoc_g_kg_avg,soc_change_mg
0,21.0,1500.0,5.0,5.0,48.393333,17.39,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5,3.31
1,21.0,1500.0,15.0,15.0,43.580000,15.57,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5,-3.03
2,21.0,1500.0,40.0,40.0,43.580000,55.38,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5,-24.46
3,21.0,1500.0,80.0,80.0,36.720000,36.10,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5,0.13
4,21.0,1500.0,5.0,5.0,48.393333,16.28,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5,0.15


In [136]:
data_soc_mg.to_csv('df_nums_soc_mg.csv')

Creating another df for soc_gkg

In [137]:
data_soc_gkg = data_imputed.copy()
data_soc_gkg.drop(['soc_mg_ha_current', 'soc_g_kg_current', "soc_mg_ha_current", 'id', "soc_change_mg"], axis=1, inplace =True)
data_soc_gkg.head()

,temp_c,precipitation_mm,depth_midpoint_previous,depth_midpoint_current,soc_g_kg_previous,soc_mg_ha_previous,latitud,longitud,elevation,slope,slopeclass,aspect,hillshade,rougness,cwd,pt10,avg_percipitation,bd_avg,ph_avg,cat_avg,clay_avg,slit_avg,sand_avg,gridsoc_g_kg_avg,soc_change_gkg
0,21.0,1500.0,5.0,5.0,48.393333,17.39,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5,0.446667
1,21.0,1500.0,15.0,15.0,43.580000,15.57,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5,-0.206667
2,21.0,1500.0,40.0,40.0,43.580000,55.38,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5,-0.206667
3,21.0,1500.0,80.0,80.0,36.720000,36.10,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5,-1.066667
4,21.0,1500.0,5.0,5.0,48.393333,16.28,-21.35,-47.07,622.0,90.0,3.0,329.53,158.33,33.97,-320.33,45.65,261.0,1347.5,52.0,11.0,35.5,19.5,45.0,16.5,0.446667


In [138]:
data_soc_gkg.to_csv('df_nums_soc_gkg.csv')